In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests
import statsmodels.stats.multicomp as multicomp

In [2]:
data_df = pd.read_excel('../data/42255_2025_1311_MOESM5_ESM.xlsx', sheet_name='2M')

In [3]:
data_df

,ND,HFD,HFD+SAL,HFD+SANA
0,39.455600,365.99760,16.04199,242.305400
1,18.783640,865.12960,90.10347,40.000000
2,22.762350,338.48630,139.11480,0.106636
3,6.562436,673.01770,685.15240,20.322560
4,87.837930,580.70030,329.86830,3.583842
5,34.872360,531.47200,16.17299,119.091800
6,45.623540,753.15520,58.87864,15.663800
7,104.736500,507.48310,381.75710,39.618500
8,45.783260,287.40850,430.83960,117.039300
9,5.236543,525.90920,146.00020,67.249050


In [4]:
data_df.columns

Index(['ND', 'HFD', 'HFD+SAL ', 'HFD+SANA'], dtype='object')

In [6]:
grp1, grp2, grp3, grp4 = data_df['ND'].dropna().values, \
data_df['HFD'].dropna().values, \
data_df['HFD+SAL '].dropna().values, \
data_df['HFD+SANA'].dropna().values

In [7]:
collect = [grp1, grp2, grp3, grp4]

In [8]:
def reorder_label(label):
    if label[-2:] == 'ND':
        return 'ND vs ' + label.split('vs')[0].strip()
    else:
        return label

In [9]:
for _ in [0]:

    print('N samples: ' + str(len(np.concatenate(collect))))
    print('ANOVA:')
    f = scipy.stats.f_oneway(grp1, grp2, grp3, grp4)
    print('F(3, ' + str(len(np.concatenate(collect))-4) + ') = ' + str(f.statistic) + ', p = ' + str(f.pvalue))

    #print('Bonferroni:')
    comp = multicomp.MultiComparison(data=np.concatenate(collect), 
                                      groups=np.concatenate([['ND']*len(grp1), 
                                                             ['HFD']*len(grp2), 
                                                             ['HFD+SAL']*len(grp3), 
                                                             ['HFD+SANA']*len(grp4)]))
    tbl, a1, a2 = comp.allpairtest(scipy.stats.ttest_ind, method= "bonf", alpha=0.05)
    bonf = pd.DataFrame(tbl)
    bonf.columns = bonf.loc[0].astype(str)
    bonf = bonf[1:]
    bonf['pval_corr'] = a1[2]
    bonf['g1'] = bonf['group1']
    bonf['g2'] = bonf['group2']
    bonf_df = bonf.copy()
    bonf_df['Bonferroni p-value'] = bonf_df['pval_corr']
    bonf_df['Comparison'] = bonf_df['g1'].astype(str) + ' vs ' + bonf_df['g2'].astype(str)
    bonf_df['Comparison'] = bonf_df['Comparison'].apply(reorder_label)

    #print('Tukey:')
    tukey = scipy.stats.tukey_hsd(*collect).pvalue
    tukey_df = pd.DataFrame({'g1':['ND', 'ND', 'ND', 'HFD', 'HFD', 'HFD+SAL'], 
     'g2':['HFD', 'HFD+SAL', 'HFD+SANA', 'HFD+SAL', 'HFD+SANA', 'HFD+SANA'],
     'Tukey p-value':[tukey[0][1], tukey[0][2], tukey[0][3], tukey[1][2], tukey[1][3], tukey[2][3]]})
    tukey_df['Comparison'] = tukey_df['g1'].astype(str) + ' vs ' + tukey_df['g2'].astype(str)

    combo_df = pd.merge(bonf_df, tukey_df, on=['Comparison'], how='outer')
    combo_df['Published p-value'] = ''
    combo_df = combo_df[['Comparison', 'Published p-value', 'Bonferroni p-value', 'Tukey p-value']].set_index('Comparison')
    print(combo_df.to_markdown())

N samples: 62
ANOVA:
F(3, 58) = 20.648111681372832, p = 3.172084955777662e-09
| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |                     |          0.236435    |     0.0155107   |
| HFD vs HFD+SANA     |                     |          1.00878e-05 |     6.69965e-08 |
| ND vs HFD           |                     |          6.9413e-06  |     3.7967e-08  |
| HFD+SAL vs HFD+SANA |                     |          0.00564841  |     0.0134063   |
| ND vs HFD+SAL       |                     |          0.00238734  |     0.007783    |
| ND vs HFD+SANA      |                     |          1           |     0.994472    |


Here is Figure 2M:

![file](../img/pub_fig_2m.png)

This is described as involving a one-way ANOVA followed by Bonferroni post-hoc. Using the [source data](https://www.nature.com/articles/s42255-025-01311-z#Sec38), we arrive at F(3, 58) = 20.6, p = 3.17E-9. However, the p-values shown for the post-hoc test are incorrect for Bonferroni post-hoc comparison (i.e., independent t-tests between groups and Bonferroni FWER correction on p-values for each pairwise test) and are a closer match for Tukey's post-hoc test. However, the p-value shown for ND vs HFD is still incorrect for Tukey's post-hoc test. See summary table below.

| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |       0.016              |          0.236    |     0.0155   |
| HFD vs HFD+SANA     |            Not shown         |          1.01e-05 |     6.70E-8 |
| ND vs HFD           |                0.00009     |          6.94E-6  |     3.797E-8  |
| HFD+SAL vs HFD+SANA |           0.013          |          0.00565  |     0.0134   |
| ND vs HFD+SAL       |             Not shown        |          0.00239  |     0.00778    |
| ND vs HFD+SANA      |              0.99       |          1.00           |     0.994    |

Could the authors clarify? 

The code for this analysis is available at [github.com/reeserich/cal_et_al_2025](https://github.com/reeserich/cal_et_al_2025).